In [ ]:
!pip install gurobipy

In [ ]:
import numpy as np
import gurobipy as gp

# Train Example

In [ ]:
objvec = [2000,2500,5000,3500]
density = [400,300,200,500]
weightmax = [10,8,12]
volmax = [5000,4000,8000]
availmax=[18,10,5,20]

In [ ]:
trainMod = gp.Model()

Trainx = trainMod.addMVar((3,4)) # tell the model how many variables there are...this can be a multi-dimensional array of variables


In [ ]:
# let's add constraints first
# we could add each constraint individually
wcons = [0]*3
wcons[0] = trainMod.addConstr(Trainx[0,0]+Trainx[0,1]+Trainx[0,2]+Trainx[0,3] <= weightmax[0])
wcons[1] = trainMod.addConstr(Trainx[1,0]+Trainx[1,1]+Trainx[1,2]+Trainx[1,3] <= weightmax[1])
wcons[2] = trainMod.addConstr(Trainx[2,0]+Trainx[2,1]+Trainx[2,2]+Trainx[2,3] <= weightmax[2])



In [ ]:
# that was a bit burdensome
# a slightly faster way to do it is with gp.quicksum
wcons = [0]*3
wcons[0] = trainMod.addConstr(gp.quicksum(Trainx[0,j] for j in range(4)) <= weightmax[0])
wcons[1] = trainMod.addConstr(gp.quicksum(Trainx[1,j] for j in range(4)) <= weightmax[1])
wcons[2] = trainMod.addConstr(gp.quicksum(Trainx[2,j] for j in range(4)) <= weightmax[2])


In [ ]:
# we can actually add all these constraints at the same time using addConstrs
wcons = trainMod.addConstrs((gp.quicksum(Trainx[i,j] for j in range(4))<=weightmax[i]) for i in range(3))


In [ ]:
# volume constraints
vcons = trainMod.addConstrs((gp.quicksum(density[j]*Trainx[i,j] for j in range(4))<=volmax[i]) for i in range(3))
# availability constraints
acons = trainMod.addConstrs((gp.quicksum(Trainx[i,j] for i in range(3))<=availmax[j]) for j in range(4))


In [ ]:
# objective
trainMod.setObjective(gp.quicksum(objvec[j]*Trainx[i,j] for i in range(3) for j in range(4)),gp.GRB.MAXIMIZE)

In [ ]:
trainMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
trainMod.optimize()

In [ ]:
Trainx.x

In [ ]:
trainMod.objVal